In [1]:
import numpy as np
import pandas as pd
df=pd.read_csv(r"C:\Users\23059\OneDrive\Desktop\Amiira\Y3S1\fyp\FraudDetectionData.csv")

In [2]:
df.shape

(6362620, 11)

In [3]:
import pandas as pd

# Count the number of 0s and 1s in the 'isFraud' column
fraud_counts = df['isFraud'].value_counts()

# Print the result
print(fraud_counts)

0    6354407
1       8213
Name: isFraud, dtype: int64


In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df['type'])
label
df.drop("type", axis=1, inplace=True)
df["type"] = label

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df['nameDest'])
label
df.drop("nameDest", axis=1, inplace=True)
df["nameDest"] = label

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df['nameOrig'])
label
df.drop("nameOrig", axis=1, inplace=True)
df["nameOrig"] = label

In [5]:
X = df.drop('isFraud', axis=1)
# Separate the target variable
y = df['isFraud']

In [6]:
from imblearn.over_sampling import SMOTE

#Upsampling via SMOTE
smote = SMOTE(sampling_strategy=0.55, random_state=0)

# Fit and apply the resampler to the entire dataset
X_resampled, y_resampled = smote.fit_resample(X, y)

In [7]:
counts = y_resampled.value_counts()
print(counts)

0    6354407
1    3494923
Name: isFraud, dtype: int64


## Reservoir sampling without replacement

In [8]:
import csv
import random

# number of data points to sample from original dataset
k= 1500000

def reservoir_sampling(iterable, k, header=True):
    reservoir = []
    for i, item in enumerate(iterable):
        if i < k:
            reservoir.append(item)
        else:
            j = random.randint(0, i)
            if j < k:
                reservoir[j] = item
    return reservoir

# Open the input CSV file
with open(r"C:\Users\23059\OneDrive\Desktop\Amiira\Y3S1\fyp\FraudDetectionData.csv") as f:
    # Check if header line exists
    header = True
    first_line = f.readline()
    if not first_line.startswith('step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud'):
        header = False
        f.seek(0)  # Rewind file pointer to beginning

    # Sample from remaining lines
    sampled_lines = []
    for i, line in enumerate(f):
        if i < k:
            sampled_lines.append(line)
        else:
            j = random.randint(0, i)
            if j < k:
                sampled_lines[j] = line

# Open the output CSV file and write the subsample to it
with open(r"C:\Users\23059\OneDrive\Desktop\Amiira\Y3S1\fyp\transfer_learning.csv", mode='w', newline='') as f:
    writer = csv.writer(f)
    if header:
        writer.writerow(first_line.strip().split(','))
    for line in sampled_lines:
        writer.writerow(line.strip().split(','))

## Pre-process larger dataset

In [9]:
df_sample_big=pd.read_csv(r"C:\Users\23059\OneDrive\Desktop\Amiira\Y3S1\fyp\transfer_learning.csv")

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df_sample_big['type'])
label
df_sample_big.drop("type", axis=1, inplace=True)
df_sample_big["type"] = label

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df_sample_big['nameDest'])
label
df_sample_big.drop("nameDest", axis=1, inplace=True)
df_sample_big["nameDest"] = label

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df_sample_big['nameOrig'])
label
df_sample_big.drop("nameOrig", axis=1, inplace=True)
df_sample_big["nameOrig"] = label

In [11]:
from sklearn.model_selection import train_test_split

X = df_sample_big.drop('isFraud', axis=1)
# Separate the target variable
y = df_sample_big['isFraud']

# Print class distribution before split
print(y.value_counts(normalize=True))

from sklearn.model_selection import train_test_split

# Assuming X contains your features and y contains your target variable
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.1, stratify=y_resampled, random_state=2)

# Print class distribution after split
print(y_train.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))

0    0.998721
1    0.001279
Name: isFraud, dtype: float64
0    0.645161
1    0.354839
Name: isFraud, dtype: float64
0    0.645162
1    0.354838
Name: isFraud, dtype: float64


In [12]:
# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import RandomUnderSampler

# #Upsampling via SMOTE
# smote = SMOTE(sampling_strategy=0.2, random_state=0)

# #Downsample via RandomUnderSampler
# rus = RandomUnderSampler(sampling_strategy=0.4, random_state=0)

# #Application of the resampling methods
# X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
# X_resampled, y_resampled = rus.fit_resample(X_resampled, y_resampled)

In [13]:
from imblearn.under_sampling import TomekLinks

#Resample using TomekLinks first
tomek_links = TomekLinks(sampling_strategy='majority')
X_train_resampled, y_train_resampled = tomek_links.fit_resample(X_train, y_train)

In [14]:
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours,OneSidedSelection
# resample the output of TomekLinks using EditedNearestNeighbours
enn = EditedNearestNeighbours(sampling_strategy='majority')
X_train_resampled_new, y_train_resampled_new = enn.fit_resample(X_train_resampled, y_train_resampled)

In [15]:
from imblearn.under_sampling import OneSidedSelection
# resample the output of EditedNearestNeighbours using One-Sided Selection
oss = OneSidedSelection(sampling_strategy='majority')
X_train_resampled_final, y_train_resampled_final = oss.fit_resample(X_train_resampled_new, y_train_resampled_new)

In [16]:
# import the necessary libraries
import pandas as pd
import numpy as np
# convert X_test to a pandas dataframe
X_test = pd.DataFrame(X_test, columns=X_train.columns)
# define a function to replace outliers with MAD for a single column
def replace_outliers_with_mad(column):
    median = np.median(column)
    mad = np.median(np.abs(column - median))
    threshold = 2.5 * mad
    column[np.abs(column - median) > threshold] = median
    return column

# apply the function to all columns of X_train_resampled_final
for i in range(X_train_resampled_final.shape[1]):
    X_train_resampled_final.iloc[:, i] = replace_outliers_with_mad(X_train_resampled_final.iloc[:, i])

# apply the function to all columns of X_test
for i in range(X_test.shape[1]):
    X_test.iloc[:, i] = replace_outliers_with_mad(X_test.iloc[:, i])

# convert the numpy arrays back to pandas dataframes
X_train_resampled_final = pd.DataFrame(X_train_resampled_final, columns=X_train.columns)
X_test = pd.DataFrame(X_test, columns=X_test.columns)

# print the modified dataframes
print(X_train_resampled_final)
print(X_test)

C:\Users\23059\anaconda3\envs\test2\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


         step         amount  oldbalanceOrg  newbalanceOrig  oldbalanceDest  \
0         259  218858.310000       0.000000             0.0     5170.999621   
1         144    9664.660000     135.000000             0.0        0.000000   
2         212    3535.920000  101588.000000             0.0        0.000000   
3         284   18362.990000   25965.350000             0.0     5170.999621   
4         349  137308.300000       0.000000             0.0     5170.999621   
...       ...            ...            ...             ...             ...   
8826762    93  137308.300000   62800.000000             0.0     5170.999621   
8826763   285  137308.300000   62800.000000             0.0       32.745541   
8826764   327  137308.300000   62800.000000             0.0     5170.999621   
8826765   323  100140.215276  100140.215276             0.0        0.000000   
8826766   409  165893.714023  165893.714023             0.0        0.000000   

         newbalanceDest  isFlaggedFraud  type  name

In [ ]:
# from sklearn.preprocessing import StandardScaler

# # fit scaler on training data
# scaler = StandardScaler()
# scaler.fit(X_train_resampled_final)

# # transform training data
# X_train_resampled_final = scaler.transform(X_train_resampled_final)

# # transform validation and test data using the same scaler
# X_test_scaled = scaler.transform(X_test)

In [ ]:
# import module
from sklearn.preprocessing import StandardScaler

# compute required values
scaler = StandardScaler()
model = scaler.fit(X_train_resampled_final)
X_train_resampled_final = model.transform(X_train_resampled_final)

In [ ]:
# compute required values
scaler = StandardScaler()
model = scaler.fit(X_test)
X_test = model.transform(X_test)

In [17]:
df_sample_big = df_sample_big.reset_index(drop=True)

### Big dataset: Pre-train CNN

KeyError: "None of [Int64Index([      0,       4,       5,       6,       8,       9,      10,\n                 11,      13,      16,\n            ...\n            8826745, 8826748, 8826749, 8826750, 8826751, 8826752, 8826756,\n            8826761, 8826763, 8826764],\n           dtype='int64', length=4413383)] are in the [columns]"

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)

In [ ]:
conv_layer = model.layers[0]
batch_layer = model.layers[1]
conv2_layer = model.layers[2]
batch2_layer = model.layers[3]
maxpooling_layer = model.layers[4]
flatten_layer = model.layers[5]

In [ ]:
model.save('model.h5')

## Go back to smaller dataset

In [ ]:
import numpy as np
import pandas as pd
df_sample=pd.read_csv(r"C:\Users\23059\OneDrive\Desktop\Amiira\Y3S1\fyp\sample.csv")

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df_sample['type'])
label
df_sample.drop("type", axis=1, inplace=True)
df_sample["type"] = label

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df_sample['nameDest'])
label
df_sample.drop("nameDest", axis=1, inplace=True)
df_sample["nameDest"] = label

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
label = le.fit_transform(df_sample['nameOrig'])
label
df_sample.drop("nameOrig", axis=1, inplace=True)
df_sample["nameOrig"] = label

In [ ]:
X = df_sample.drop('isFraud', axis=1)
# Separate the target variable
y = df_sample['isFraud']

In [ ]:
from imblearn.over_sampling import SMOTE

#Upsampling via SMOTE
smote = SMOTE(sampling_strategy=0.55, random_state=0)

# Fit and apply the resampler to the entire dataset
X_resampled, y_resampled = smote.fit_resample(X, y)

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming X contains your features and y contains your target variable
X_train, X_test, y_train, y_test = train_test_split(X_resampled,  y_resampled, test_size=0.1, stratify= y_resampled, random_state=2)

# Print class distribution after split
print(y_train.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))

In [ ]:
from imblearn.under_sampling import TomekLinks

#Resample using TomekLinks first
tomek_links = TomekLinks(sampling_strategy='majority')
X_train_resampled, y_train_resampled = tomek_links.fit_resample(X_train, y_train)

In [ ]:
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours,OneSidedSelection
# resample the output of TomekLinks using EditedNearestNeighbours
enn = EditedNearestNeighbours(sampling_strategy='majority')
X_train_resampled_new, y_train_resampled_new = enn.fit_resample(X_train_resampled, y_train_resampled)

In [ ]:
from imblearn.under_sampling import OneSidedSelection
# resample the output of EditedNearestNeighbours using One-Sided Selection
oss = OneSidedSelection(sampling_strategy='majority')
X_train_resampled_final, y_train_resampled_final = oss.fit_resample(X_train_resampled_new, y_train_resampled_new)

In [ ]:
df_sample = df_sample.reset_index(drop=True)

In [ ]:
# import the necessary libraries
import pandas as pd
import numpy as np
# convert X_test to a pandas dataframe
X_test = pd.DataFrame(X_test, columns=X_train.columns)
# define a function to replace outliers with MAD for a single column
def replace_outliers_with_mad(column):
    median = np.median(column)
    mad = np.median(np.abs(column - median))
    threshold = 2.5 * mad
    column[np.abs(column - median) > threshold] = median
    return column

# apply the function to all columns of X_train_resampled_final
for i in range(X_train_resampled_final.shape[1]):
     X_train_resampled_final.iloc[:, i] = replace_outliers_with_mad(X_train_resampled_final.iloc[:, i])
   # X_train_resampled_final[:, i] = replace_outliers_with_mad(X_train_resampled_final[:, i])

# apply the function to all columns of X_test
for i in range(X_test.shape[1]):
    X_test.iloc[:, i] = replace_outliers_with_mad(X_test.iloc[:, i])

# convert the numpy arrays back to pandas dataframes
X_train_resampled_final = pd.DataFrame(X_train_resampled_final, columns=X_train.columns)
X_test = pd.DataFrame(X_test, columns=X_test.columns)

# print the modified dataframes
print(X_train_resampled_final)
print(X_test)

In [ ]:
# # import module
# from sklearn.preprocessing import StandardScaler

# # compute required values
# scaler = StandardScaler()
# model = scaler.fit(X_train_resampled_final)
# X_train_resampled_final = model.transform(X_train_resampled_final)

In [ ]:
# # compute required values
# scaler = StandardScaler()
# model = scaler.fit(X_test)
# X_test = model.transform(X_test)

In [ ]:
import numpy as np
import tensorflow as tf
from keras.regularizers import l1
from tensorflow.keras.layers import Reshape, LSTM, Dense, TimeDistributed, Conv1D, BatchNormalization, MaxPooling1D, Flatten

encoding_dim = 6
num_features = 10 # number of input variables

# Define the autoencoder to encode each feature
input_shape = (10,)  # N is the number of input features

encoder = tf.keras.layers.Dense(units=10, activation='elu', input_shape=input_shape)
first_hidden_layer = tf.keras.layers.Dense(units=encoding_dim, activation='elu')
decoder = tf.keras.layers.Dense(units=10, activation='sigmoid')

autoencoder = tf.keras.Sequential([encoder, first_hidden_layer, decoder])

# Load the pre-trained CNN
CNN = tf.keras.models.load_model('model.h5')

# Freeze all trainable layers in CNN
CNN.trainable = False

V = np.zeros((num_features, encoding_dim))

# 1st for loop: Encoding
for i in range(num_features):
    V[i] = autoencoder.predict(X_train_resampled_final[i:i+1])[0][0:encoding_dim]

import numpy as np

# Reshape the input data to match the expected input shape of the CNN model
V = np.reshape(V, (1, 1, num_features, encoding_dim))
V = np.transpose(V, (0, 1, 3, 2))
V = np.reshape(V, (1, -1, num_features, 1))

# Pass the input data to the pre-trained CNN model
C = CNN(V)

# Reshape the output of the CNN layer to match the expected input shape of the LSTM layer
C = np.reshape(C, (1, C.shape[1], C.shape[2]))

# Add the LSTM layers to the model
LSTM_model = tf.keras.Sequential()
LSTM1 = LSTM(units=128, return_sequences=True, input_shape=(None, num_features*encoding_dim+1))
LSTM_model.add(LSTM1)

# Add a fully connected layer
LSTM_model.add(Dense(units=20, activation='tanh', kernel_regularizer=l1(0.00114)))

# Add a dropout layer
LSTM_model.add(tf.keras.layers.Dropout(0.1))

# Add the output layer
LSTM_model.add(Dense(units=1, activation='sigmoid'))

# Define the learning rate
learning_rate = 0.0015

# Create the Adam optimizer with the defined learning rate
opt_LSTM = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compile the model
LSTM_model.compile(optimizer=opt_LSTM, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.utils import plot_model
plot_model(LSTM_model, to_file='LSTM_model.png', show_shapes=True)

In [ ]:
LSTM_model.summary()

In [ ]:
# from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, MaxPooling1D, Flatten, LSTM, Concatenate, TimeDistributed, Dense

# cnn_input = Input(shape=(None, 10, 1))
# cnn_layer = TimeDistributed(Conv1D(64, kernel_size=3, activation='relu'))(cnn_input)
# cnn_layer = TimeDistributed(BatchNormalization())(cnn_layer)
# cnn_layer = TimeDistributed(Conv1D(64, kernel_size=3, activation='relu'))(cnn_layer)
# cnn_layer = TimeDistributed(BatchNormalization())(cnn_layer)
# cnn_layer = TimeDistributed(MaxPooling1D(pool_size=2))(cnn_layer)
# cnn_layer = TimeDistributed(Flatten())(cnn_layer)

# # lstm_input = Input(shape=(None, num_features*encoding_dim+1))
# lstm_layer = LSTM(units=128, return_sequences=True)(cnn_layer)

# concat_layer = Concatenate()([cnn_layer, lstm_layer])
# dense_layer = Dense(64)(concat_layer)
# output_layer = Dense(1)(dense_layer)

# model = Model(inputs=[cnn_input, lstm_input], outputs=output_layer)

In [ ]:
conv_layer = model.layers[0]
batch_layer = model.layers[1]
conv2_layer = model.layers[2]
batch2_layer = model.layers[3]
maxpooling_layer = model.layers[4]
flatten_layer = model.layers[5]

from keras.models import Model

# cnn_input = Input(shape=(None, 10, 1))
# cnn_layer = TimeDistributed(Conv1D(64, kernel_size=3, activation='relu'))(cnn_input)
# cnn_layer = TimeDistributed(BatchNormalization())(cnn_layer)
# cnn_layer = TimeDistributed(Conv1D(64, kernel_size=3, activation='relu'))(cnn_layer)
# cnn_layer = TimeDistributed(BatchNormalization())(cnn_layer)
# cnn_layer = TimeDistributed(MaxPooling1D(pool_size=2))(cnn_layer)
# cnn_layer = TimeDistributed(Flatten())(cnn_layer)

reshape_layer = Reshape((-1, flatten_layer.output_shape[-1]))(flatten_layer.output)
lstm_layer = LSTM(64)(reshape_layer)
dense_layer = Dense(32, activation='relu')(lstm_layer)
output_layer = Dense(1, activation='sigmoid')(dense_layer)

new_model = Model(inputs=model.inputs, outputs=output_layer)

In [ ]:
from tensorflow.keras.optimizers import Adam

# Define the learning rate
learning_rate = 0.0015

# Create the Adam optimizer with the defined learning rate
optimizer = Adam(lr=learning_rate)

# Compile the model
new_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Fit the model
new_model.fit(X_train_resampled_final, y_train_resampled_final, epochs=10, batch_size=32)

In [ ]:
new_model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(new_model, to_file='new_model.png', show_shapes=True)

In [ ]:
# from tensorflow.keras.optimizers import Adam

# # Define the learning rate
# learning_rate = 0.0015

# # Create the Adam optimizer with the defined learning rate
# optimizer = Adam(lr=learning_rate)

# # Compile the model
# new_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# # Fit the model
# new_model.fit([X_cnn_input, X_lstm_input], y_train_resampled_final, epochs=1, batch_size=32)

In [ ]:
# from sklearn.metrics import f1_score, precision_score, recall_score

# # Get predictions on X_test
# y_pred = new_model.predict([X_test_cnn_input, X_test_lstm_input])

# # Convert probabilities to binary predictions
# y_pred_binary = (y_pred > 0.5).astype(int)

# # Calculate evaluation metrics
# f1 = f1_score(y_test, y_pred_binary)
# precision = precision_score(y_test, y_pred_binary)
# recall = recall_score(y_test, y_pred_binary)

# print('F1 score:', f1)
# print('Precision:', precision)
# print('Recall:', recall)

In [ ]:
X_test_cnn_input = np.expand_dims(X_test, axis=2) # shape: (None, 10, 1)
X_test_lstm_input = np.expand_dims(X_test, axis=1) # shape: (None, 1, 10)
X_test_lstm_input = np.repeat(X_test_lstm_input, 61, axis=1) # shape: (None, 61, 10)

y_pred = new_model.predict([X_test_cnn_input, X_test_lstm_input])

In [ ]:
X_test_cnn_input = np.expand_dims(X_test, axis=2) # shape: (None, 10, 1)
X_test_lstm_input = np.expand_dims(X_test, axis=1) # shape: (None, 1, 10)
X_test_lstm_input = np.transpose(X_test_lstm_input, (0, 2, 1)) # shape: (None, 10, 1)
X_test_lstm_input = np.repeat(X_test_lstm_input, 61, axis=2) # shape: (None, 10, 61)

y_pred = new_model.predict([X_test_cnn_input, X_test_lstm_input])

In [ ]:
# from sklearn.metrics import roc_curve, auc
# import matplotlib.pyplot as plt

# # Generate predicted probabilities for the test set
# y_pred_prob = model.predict(X_test)

# # Compute ROC curve and ROC area for each class
# fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
# roc_auc = auc(fpr, tpr)

# # Plot ROC curve
# plt.figure()
# plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic')
# plt.legend(loc="lower right")
# plt.show()